# VIC - Introduction to Visual Computing

## Low-Level Vision, Image Filtering, Edge Detection.

During these exercises, you will become familiar with image processing and low level vision in `Python`.

For the lab exercises use `jupyter` (or anything else at your own risk). It can be installed by issuing `pip install jupyter` on Linux after installing pip itself with `apt install python-pip`. Mac and Windows users can find the installation instructions online. The first thing that is needed is to import some libraries, such as `numpy`, `scipy`, `random` and `matplotlib` for visualization. If they are missing you can simply install them with e.g., `pip install numpy`

In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

### Exercise 1

Consider the filter `f = [1,3,2]` and the 1D image `I = [0,1,2,3,3,3,1,3,6]`. What is the result of `f*I`? Implement your own convolution and then compare it to `scipy.ndimage.convolve1d`. Pad the image with zeros at the boundaries if necessary. Plot the two signals and observe their difference. What does this filter do?

In [ ]:
f = np.array([1, 3, 2])
I = np.array([0, 1, 2, 3, 3, 3, 1, 3, 6])

In [ ]:
O = np.zeros_like(I)
pad_I = np.pad(I, pad_width=1, mode='constant')

for i in range(len(I)):
    O[i] = np.sum(pad_I[i:i+3] * f[::-1])

print(O)

In [ ]:
from scipy.ndimage import convolve1d

O_np = convolve1d(I, f, mode='constant')
print(O_np)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))

ax.plot(O_np, color='orange')
ax.scatter(range(len(O_np)), O_np, marker='o', color='orange')
ax.plot(I, color='blue')
ax.scatter(range(len(I)), I, marker='o', color='blue')

### Exercise 2

Now let us pass to two dimensions. We read an image and the corresponding (human generated) ground truth of the image contours.

In [ ]:
from scipy import io

mat = io.loadmat('12003.mat')
bound = mat['groundTruth'][0, 2][0][0][1]

image = plt.imread('12003.jpg')

fig, axes = plt.subplots(figsize=(15, 10), ncols=2)

axes[0].imshow(image)
axes[1].imshow(bound, cmap='gray')

The exercises below will be simplified if we work on a grayscale representation. We also normalise to a [0, 1] range

In [ ]:
# conversion to grayscale
r, g, b = image[:,:,0], image[:,:,1], image[:,:,2]
img = (0.2989 * r + 0.5870 * g + 0.1140 * b) / 255.

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img, cmap='gray')

### Exercise 3

Salt and pepper noise randomly sets each pixel to the minimum or maximum of the image range (here, 0 or 1). Implement a function with parameter $p$ to add salt and pepper noise to the grayscale `img`. Each pixel should become 0 with probability $p/2$, 1 with probability $p/2$, and stay unchanged with probaility $1 - p$. Use the `random.random` function.

In [ ]:
from random import random

def salt_and_pepper_noise(img, p):
    """
    Adds salt and pepper noise to the image
    p: probability of the noise
    """
    assert 0 <= p and p <= 1  # valid probability
    assert len(img.shape) == 2  # grayscale image
    h, w = img.shape
    
    I_out = img.copy()

    for i in range(h):
        for j in range(w):
            if random() < p:
                if random() < 0.5: 
                    I_out[i, j] = 0  # pepper
                else:
                    I_out[i, j] = 1  # salt
    return I_out

We can plot the result. Try adjusting the amount of salt and pepper. For best results, proceed to the next exercise with $p \approx 0.1$

In [ ]:
sp_img = salt_and_pepper_noise(img, 0.1)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(sp_img, cmap='gray')

## Low-pass filters

Low-pass filters attenuate high-frequency signals, and can be used for smoothing and noise removal. Here we will implement three such filters. Take some time to check the different results. Which filter behaves better and why?

### Exercise 4.1

Implement a mean filter,

$$
M = 1/9 \cdot \begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}$$

as a `np.array` object to remove the salt and pepper noise from the image. Use the `scipy.ndimage.convolve` function to convolve the filter. Try changing the filter size. What happens?

In [ ]:
from scipy.ndimage import convolve

radius = 5
MEAN_FILTER = (1 / radius ** 2) * np.ones((radius, radius))
mean_img = convolve(sp_img, MEAN_FILTER, mode='constant')

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(mean_img, cmap='gray')

### Exercise 4.2

Apply a Gaussian filter to the image. This time, use the `scipy.ndimage.gaussian_filter` function. Try adjusting the `sigma` parameter. What do you observe?

In [ ]:
from scipy.ndimage import gaussian_filter

gaussian_img = gaussian_filter(sp_img, sigma=3)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(gaussian_img, cmap='gray')

### Exercise 4.3

Implement a median filter to denoise the image. Note that this filter is non-linear, so this time you will have to code the sliding-window process yourself. Can you think of an improvement to a simple sliding-window approach?

In [ ]:
def median_filter(img, radius):
    h, w = img.shape

    I_out = img.copy()

    for i in range(h):
        for j in range(w):
            I_out[i, j] = np.median(img[max(i-radius, 0):min(i+radius, h),
                                        max(j-radius, 0):min(j+radius, w)])
    return I_out

median_img = median_filter(sp_img, 1)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(median_img, cmap='gray')

## High-pass filters

High-pass filters promote high-frequency information, and can be used for image sharpening and edge detection. Here we implement two such filters.

### Exercise 5.1

Implement the Laplacian filter,

$$\Delta I = \frac{\partial^2 I}{\partial x^2} + \frac{\partial^2 I}{\partial y^2} := I * \Bigg(\begin{bmatrix}
0 & 0 & 0 \\
-1 & 2 & -1 \\
0 & 0 & 0
\end{bmatrix} + 
\begin{bmatrix}
0 & -1 & 0 \\
0 & 2 & 0 \\
0 & -1 & 0
\end{bmatrix}\Bigg)$$

on the original image to detect the edges. Once more, you should use `scipy.ndimage.convolve` to perform the convolution. Perform the same using the smoothed images from a Gaussian filter. What do you observe?

In [ ]:
LAPLACE_FILTER = np.array([[0, -1, 0],
                           [-1, 4, -1],
                           [0, -1, 0]])

laplace_img = convolve(img, LAPLACE_FILTER, mode='constant')

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(laplace_img, cmap='gray')

### Exercise 5.2

Implement the Sobel filters and apply them to `img` to detect the edges,

$$G_x = I * \begin{bmatrix}
1 & 0 & -1 \\
2 & 0 & -2 \\
1 & 0 & -1
\end{bmatrix}, G_y = I * \begin{bmatrix}
1 & 2 & 1 \\
0 & 0 & 0 \\
-1 & -2 & -1
\end{bmatrix}.$$

In [ ]:
SOBEL_X = np.array([[+1, 0, -1],
                    [+2, 0, -2],
                    [+1, 0, -1]])

SOBEL_Y = np.array([[+1, +2, +1],
                    [0, 0, 0],
                    [-1, -1, -2]])

sobel_x_img = convolve(img, SOBEL_X, mode='constant')
sobel_y_img = convolve(img, SOBEL_Y, mode='constant')

fig, axes = plt.subplots(figsize=(15, 10), ncols=2)
axes[0].imshow(sobel_x_img, cmap='gray')
axes[0].set_title('$G_x$')
axes[1].imshow(sobel_y_img, cmap='gray')
axes[1].set_title('$G_y$')

### Exercise 5.3

Combine the Sobel gradient images to obtain the gradient magnitude image,

$$G = \sqrt{G_x^2 + G_y^2}$$

Then, binarise this by choosing a suitable threshold in order to obtain a segmentation of the image edges.

In [ ]:
magnitude_img = np.sqrt(sobel_x_img ** 2 + sobel_y_img ** 2)

# normalise image
magnitude_img -= np.min(magnitude_img)
magnitude_img /= np.max(magnitude_img)

THRESHOLD = 0.2
segmented_img = magnitude_img > THRESHOLD

fig, axes = plt.subplots(figsize=(15, 10), ncols=2)
axes[0].imshow(magnitude_img, cmap='gray')
axes[0].set_title('$G$')
axes[1].imshow(segmented_img, cmap='gray')
axes[1].set_title('$G > T$')

We can then evaluate the segmentation quality as compared with the ground truth image using `sklearn.metrics`. Try adjusting the threshold above. How do the evaluation metrics change?

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('Precision:\t %.02f' % precision_score(segmented_img.flatten(), bound.flatten()))
print('Recall:\t\t %.02f' % recall_score(segmented_img.flatten(), bound.flatten()))
print('F1 score:\t %.02f' % f1_score(segmented_img.flatten(), bound.flatten()))
print('Accuracy:\t %.02f' % accuracy_score(segmented_img.flatten(), bound.flatten()))

### Exercise 5.4

Try the different morphological operators in `scipy.ndimage` to post-process the segmented image, and improve the overall accuracy. Note, that despite the mathematical jargon, a dilation is really just a *max* filter, and an erosion is a *min* filter, akin to the median filter we coded before. The structuring element specifies the size and shape of the filter.

In [ ]:
from scipy.ndimage import binary_dilation, binary_erosion, binary_opening, binary_closing

# Define structuring element
selem = np.ones((2, 2))

dilated_image = binary_dilation(segmented_img, selem)
eroded_image = binary_erosion(segmented_img, selem)
opened_image = binary_opening(segmented_img, selem)
closed_image = binary_closing(segmented_img, selem)

fig, axes = plt.subplots(figsize=(15, 10), nrows=2, ncols=2)
axes[0][0].imshow(dilated_image, cmap='gray')
axes[0][0].set_title('Dilation')
axes[0][1].imshow(eroded_image, cmap='gray')
axes[0][1].set_title('Erosion')
axes[1][0].imshow(opened_image, cmap='gray')
axes[1][0].set_title('Opening')
axes[1][1].imshow(closed_image, cmap='gray')
axes[1][1].set_title('Closing')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('Precision:\t %.02f' % precision_score(opened_image.flatten(), bound.flatten()))
print('Recall:\t\t %.02f' % recall_score(opened_image.flatten(), bound.flatten()))
print('F1 score:\t %.02f' % f1_score(opened_image.flatten(), bound.flatten()))
print('Accuracy:\t %.02f' % accuracy_score(opened_image.flatten(), bound.flatten()))

## Fourier analysis

We will now perform the main low- and high-pass filter operations in frequency space using `scipy.fft` functions for Fourier analysis. First, let's plot the frequency representation of our grayscale image. We define a helper function `vis_fft` to visualise lower amplitude frequencies more easily. Notice, the transform is a complex-valued image. 

In [ ]:
from scipy.fft import fft2, fftshift

# Apply 2D fast Fourier transform algorithm
fft_img = fft2(img)

# Helper function for visualising low-intensity Fourier values
def vis_fft(fft):
    return np.log(np.abs(fft) + 1e-8)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(vis_fft(fftshift(fft_img)))

### Exercise 6.1

We form a 2D Gaussian $G$ in pixel space. Applying the convolution theorem,

$$\mathcal{F}(G * I) = \mathcal{F}(G) \cdot \mathcal{F}(I),$$

we multiply the frequency representation of the Gaussian $\mathcal{F}(G)$ element-wise with that of the image, and visualise each. Note we apply the `fftshift` function to centerise the low frequencies. What is the shape of $\mathcal{F}(G)$?

In [ ]:
from scipy.fft import ifft2, ifftshift
from scipy import signal

# Outer product of Gaussians
GAUSSIAN_FILTER = np.outer(signal.gaussian(img.shape[0], std=3),
                           signal.gaussian(img.shape[1], std=3))

# Transform Gaussian filter
fft_gaussian = fft2(ifftshift(GAUSSIAN_FILTER))

# By the convolution theorem!
fft_convolved = fft_img * fft_gaussian

fig, axes = plt.subplots(figsize=(15, 10), ncols=3)

axes[0].imshow(GAUSSIAN_FILTER)
axes[0].set_title('$G$')
axes[1].imshow(vis_fft(fftshift(fft_gaussian)))
axes[1].set_title('$\mathcal{F}(G)$')
axes[2].imshow(vis_fft(fftshift(fft_convolved)))
axes[2].set_title('$\mathcal{F}(G * I) = \mathcal{F}(G) * \mathcal{F}(I)$')

All that remains is to apply the inverse FFT to recover the original image! What was the effect of the Gaussian kernel? How does it compare with that of **Exercise 4.2**?

In [ ]:
restored_img = ifft2(fft_convolved).real

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(restored_img, cmap='gray')

### Exercise 6.2

Mask (set to zero) the low frequencies of `fft_img` so as perform a high-pass filter in Fourier space.

In [ ]:
h, w = fft_img.shape
r = 10

fft_low_pass = fftshift(fft_img)

for i in range(h):
    for j in range(w):
        if np.sqrt((i - h / 2) ** 2 + (j - w / 2) ** 2) < r:
            fft_low_pass[i, j] = 0

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(vis_fft(fft_low_pass))

We finally restore the image to pixel space.

In [ ]:
restored_img = ifft2(ifftshift(fft_low_pass)).real

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(restored_img, cmap='gray')